In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU, ReLU, ELU, PReLU
from tensorflow.keras.optimizers import Adam,SGD,RMSprop,Adagrad
from kerastuner.tuners import RandomSearch
import datetime


C:\Users\sahil\AppData\Local\Temp\ipykernel_6640\3225021342.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=20,
                               mode='auto',
                               restore_best_weights=False,
                               baseline=None,
                              #  min_delta = 0.0001,

                              )

checkpoint = ModelCheckpoint(
    filepath='best_model.h5',       # File name to save the model
    monitor='val_loss',             # Metric to monitor (commonly val_loss or val_accuracy)
    save_best_only=True,            # Save only when performance improves
    mode='min',                     # 'min' if you're monitoring loss; 'max' for accuracy
    save_weights_only=False,        # Set to True if you only want to save weights
    verbose=1                       # Print a message each time model is saved
)

In [3]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_of_layers',2,10)):
        if i == 0:
            model.add(Dense(
                units = hp.Int('units'+str(i),min_value = 2 , max_value = 528,step = 8),
                activation = hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid','elu','selu']),
                input_dim = 11
            ))
        else:
            model.add(Dense(
                units = hp.Int('units'+str(i),min_value = 2 , max_value = 528,step = 8),
                activation = hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid','elu','selu']),
            ))

    model.add(Dense(units=1,activation='sigmoid'))
    
    # Choose optimizer
    optimizer_name = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

    # Set optimizer-specific parameters
    if optimizer_name == 'adam':
        learning_rate = hp.Float('adam_lr', 1e-5, 1e-2, sampling='log')
        beta_1 = hp.Float('adam_beta_1', 0.8, 0.99, step=0.01)
        beta_2 = hp.Float('adam_beta_2', 0.9, 0.999, step=0.01)
        optimizer = Adam(
            learning_rate=learning_rate,
            beta_1=beta_1,
            beta_2=beta_2
        )

    elif optimizer_name == 'sgd':
        learning_rate = hp.Float('sgd_lr', 1e-5, 1e-1, sampling='log')
        momentum = hp.Float('sgd_momentum', 0.0, 0.9, step=0.1)
        nesterov = hp.Boolean('sgd_nesterov')
        optimizer = SGD(
            learning_rate=learning_rate,
            momentum=momentum,
            nesterov=nesterov
        )

    elif optimizer_name == 'rmsprop':
        learning_rate = hp.Float('rmsprop_lr', 1e-5, 1e-2, sampling='log')
        rho = hp.Float('rmsprop_rho', 0.8, 0.99, step=0.01)
        momentum = hp.Float('rmsprop_momentum', 0.0, 0.9, step=0.1)
        optimizer = RMSprop(
            learning_rate=learning_rate,
            rho=rho,
            momentum=momentum
        )

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

    
            

In [4]:
tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=15,
    directory = 'project',
    project_name = 'churn'
)

d:\Ann PROJECT\mysvenv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
import dill
import pandas as pd

df = pd.read_csv('Churn_Modelling.csv')

with open('model.m5','rb') as f:
    model = dill.load(f)


transformed_data = model.transform(df)

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(transformed_data,df.iloc[:,-1],test_size=0.3,random_state=43)

In [7]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 15 Complete [00h 00m 07s]
val_accuracy: 0.8293333053588867

Best val_accuracy So Far: 0.8650000095367432
Total elapsed time: 00h 02m 31s


In [8]:
bestmodel = tuner.get_best_models(num_models=1)[0]

d:\Ann PROJECT\mysvenv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Ann PROJECT\mysvenv\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
bestmodel.fit(X_train,y_train,validation_data=[X_test,y_test],callbacks=[early_stopping,checkpoint],initial_epoch=11,epochs = 100)

Epoch 12/100
208/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8517 - loss: 0.3595
Epoch 12: val_loss improved from inf to 0.35985, saving model to best_model.h5


219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8514 - loss: 0.3598 - val_accuracy: 0.8583 - val_loss: 0.3599
Epoch 13/100
203/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8561 - loss: 0.3510
Epoch 13: val_loss improved from 0.35985 to 0.34095, saving model to best_model.h5


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8557 - loss: 0.3515 - val_accuracy: 0.8643 - val_loss: 0.3409
Epoch 14/100
186/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8574 - loss: 0.3516
Epoch 14: val_loss did not improve from 0.34095
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8572 - loss: 0.3515 - val_accuracy: 0.8603 - val_loss: 0.3507
Epoch 15/100
216/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8556 - loss: 0.3441
Epoch 15: val_loss improved from 0.34095 to 0.33477, saving model to best_model.h5


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8556 - loss: 0.3442 - val_accuracy: 0.8670 - val_loss: 0.3348
Epoch 16/100
184/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8605 - loss: 0.3464
Epoch 16: val_loss did not improve from 0.33477
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8596 - loss: 0.3473 - val_accuracy: 0.8617 - val_loss: 0.3520
Epoch 17/100
208/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8580 - loss: 0.3441
Epoch 17: val_loss did not improve from 0.33477
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8578 - loss: 0.3447 - val_accuracy: 0.8540 - val_loss: 0.3739
Epoch 18/100
212/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8629 - loss: 0.3437
Epoch 18: val_loss did not improve from 0.33477
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8627 - loss: 0.3441 - val_accuracy: 0.8643 - val_loss: 0.3544
Epoch 19/100
196/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8576 - loss: 0.3533
Epoch 19: val_loss did not improve f

In [10]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the Keras model
loaded_model = load_model('best_model.h5')

In [12]:
y_pred = loaded_model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
y_train_pred = loaded_model.predict(X_train)
y_train_pred_binary = (y_train_pred > 0.5).astype(int)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step


In [13]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [14]:
accuracy = accuracy_score(y_test,y_pred_binary)
accuracy_train = accuracy_score(y_train,y_train_pred_binary)

In [15]:
print('test accuracy :' ,accuracy)
print('train accuracy :' ,accuracy_train)

test accuracy : 0.867
train accuracy : 0.8627142857142858


In [16]:
bestmodel.evaluate(X_train,y_train)

219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8670 - loss: 0.3304


[0.3356480896472931, 0.8612856864929199]